# Building an Anime Recommender System

In [1]:
import sys
sys.path.append("..")

from animerec import models
from animerec import utils

import pandas
import pickle

### 1. Load Dataset and Model

In [2]:
# Load anime metadata.
pickle_in = open("../data/animes.pickle", 'rb')
animes = pickle.load(pickle_in)
animes.index = animes['anime_id']
animes = animes[ ['title', 'title_english', 'type', 'premiered', 'genre'] ]
pickle_in.close()

# Load model trained on server.
pickle_in = open("../model/model.pickle", 'rb')
model = pickle.load(pickle_in)
pickle_in.close()

# Load anime id's.
pickle_in = open("../model/item-id.pickle", 'rb')
item_id = pickle.load(pickle_in)
pickle_in.close()

### 2. Webscrape current profile and create prediction

In [3]:
myprofile = utils.get_user_anime_list('Manuel')
x = utils.get_score_vector_from_user_anime_list(myprofile, item_id)

Scraped 300 additional anime from https://myanimelist.net/animelist/Manuel/load.json?offset=0&status=7
Scraped 177 additional anime from https://myanimelist.net/animelist/Manuel/load.json?offset=300&status=7


In [4]:
rhat = model.predict(x)

In [9]:
prediction = utils.prediction_to_dataframe(rhat, myprofile, item_id, keep_all=False)
prediction = pandas.DataFrame(prediction)
prediction.columns = ['score']
prediction = prediction.join(animes)

In [10]:
prediction.sort_values(by='score', ascending=False).head(60)

,score,title,title_english,type,premiered,genre
440,9.870317,Shoujo Kakumei Utena,Revolutionary Girl Utena,TV,Spring 1997,"Mystery, Comedy, Psychological, Drama, Fantasy..."
1,9.680458,Cowboy Bebop,Cowboy Bebop,TV,Spring 1998,"Action, Adventure, Comedy, Drama, Sci-Fi, Space"
721,9.581140,Princess Tutu,Princess Tutu,TV,Summer 2002,"Comedy, Drama, Magic, Romance, Fantasy"
820,9.557186,Ginga Eiyuu Densetsu,Legend of the Galactic Heroes,OVA,NaN,"Military, Sci-Fi, Space, Drama"
339,9.431958,Serial Experiments Lain,Serial Experiments Lain,TV,Summer 1998,"Dementia, Drama, Mystery, Psychological, Sci-F..."
3297,9.428207,Aria The Origination,Aria The Origination,TV,Winter 2008,"Fantasy, Sci-Fi, Shounen, Slice of Life"
17074,9.378211,Monogatari Series: Second Season,Monogatari Series: Second Season,TV,Summer 2013,"Mystery, Comedy, Supernatural, Romance, Vampire"
35247,9.317616,Owarimonogatari 2nd Season,Owarimonogatari Second Season,TV,Summer 2017,"Comedy, Mystery, Supernatural, Vampire"
145,9.316278,Kareshi Kanojo no Jijou,His and Her Circumstances,TV,Fall 1998,"Comedy, Drama, Romance, School, Shoujo, Slice ..."
1453,9.296081,Maison Ikkoku,Maison Ikkoku,TV,Spring 1986,"Slice of Life, Comedy, Drama, Romance, Seinen"
